In [7]:
### Add lines to import modules as needed
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import Input, layers
from tensorflow.keras import backend as K


# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import Input, layers

In [8]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

# Now separate out a validation set.
val_frac = 0.1
num_val_samples = int(len(train_images)*val_frac)
# choose num_val_samples indices up to the size of train_images, !replace => no repeats
val_idxs = np.random.choice(np.arange(len(train_images)), size=num_val_samples, replace=False)
trn_idxs = np.setdiff1d(np.arange(len(train_images)), val_idxs)
val_images = train_images[val_idxs, :,:,:]
train_images = train_images[trn_idxs, :,:,:]

val_labels = train_labels[val_idxs]
train_labels = train_labels[trn_idxs]

train_labels = train_labels.squeeze()
test_labels = test_labels.squeeze()
val_labels = val_labels.squeeze()

input_shape  = train_images.shape[1:]
train_images = train_images / 255.0
test_images  = test_images  / 255.0
val_images   = val_images   / 255.0

In [11]:
def build_model3():
    input_shape = (32, 32, 3)

    # Define the input tensor
    inputs = tf.keras.Input(shape=input_shape)

    # First convolution block
    x = tf.keras.layers.Conv2D(32, kernel_size=(3,3), strides=(2,2), activation="relu", padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)

    # Second convolution block with residual connection
    x = tf.keras.layers.Conv2D(64, kernel_size=(3,3), strides=(2,2), activation="relu", padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    residual = x
    x = tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation="relu", padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.add([x, residual])
    
    # Third convolution block with residual connection
    x = tf.keras.layers.Conv2D(128, kernel_size=(3,3), strides=(2,2), activation="relu", padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    residual = tf.keras.layers.Conv2D(128, kernel_size=(1,1), strides=(2,2), padding='same')(residual)
    residual = tf.keras.layers.BatchNormalization()(residual)
    x = tf.keras.layers.add([x, residual])
    x = tf.keras.layers.Conv2D(128, kernel_size=(3,3), activation="relu", padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Conv2D(128, kernel_size=(3,3), activation="relu", padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)


    # Fourth convolution block with residual connection
    x = tf.keras.layers.Conv2D(128, kernel_size=(3,3), activation="relu", padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    residual = x
    x = tf.keras.layers.Conv2D(128, kernel_size=(3,3), activation="relu", padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.add([x, residual])

    # Finishing layers
    x = tf.keras.layers.MaxPooling2D(pool_size=(4, 4), strides=(4, 4))(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(10)(x)

    # Define the model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # Compile the model
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    return model

In [12]:
model3 = build_model3()
model3.compile(optimizer='adam',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])
model3.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_8 (Conv2D)              (None, 16, 16, 32)   896         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_6 (BatchNo  (None, 16, 16, 32)  128         ['conv2d_8[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 dropout_4 (Dropout)            (None, 16, 16, 32)   0           ['batch_normalization_6[0][0]

 dense_1 (Dense)                (None, 10)           1290        ['dropout_12[0][0]']             
                                                                                                  
Total params: 750,346
Trainable params: 748,490
Non-trainable params: 1,856
__________________________________________________________________________________________________


In [13]:
model3.fit(train_images, train_labels,
        validation_data=(val_images, val_labels),
        epochs=50)

Epoch 1/50


2023-02-20 21:56:04.491907: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout_4/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-02-20 21:56:05.187361: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8201
2023-02-20 21:56:07.169294: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-02-20 21:56:07.184982: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f21e9a12e10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-20 21:56:07.185022: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2023-02-20 21:56:07.189020: I tensorflow/compiler/mlir/tensorflow/utils/dump_

1407/1407 [==============================] - 18s 9ms/step - loss: 1.8231 - accuracy: 0.3465 - val_loss: 1.5772 - val_accuracy: 0.4432
Epoch 2/50
1407/1407 [==============================] - 12s 9ms/step - loss: 1.4772 - accuracy: 0.4659 - val_loss: 1.3676 - val_accuracy: 0.4948
Epoch 3/50
1407/1407 [==============================] - 12s 9ms/step - loss: 1.3257 - accuracy: 0.5265 - val_loss: 1.1337 - val_accuracy: 0.5972
Epoch 4/50
1407/1407 [==============================] - 13s 9ms/step - loss: 1.2060 - accuracy: 0.5720 - val_loss: 1.1013 - val_accuracy: 0.6030
Epoch 5/50
1407/1407 [==============================] - 12s 9ms/step - loss: 1.1110 - accuracy: 0.6092 - val_loss: 1.1494 - val_accuracy: 0.6042
Epoch 6/50
1407/1407 [==============================] - 12s 9ms/step - loss: 1.0358 - accuracy: 0.6394 - val_loss: 1.0808 - val_accuracy: 0.6162
Epoch 7/50
1407/1407 [==============================] - 12s 9ms/step - loss: 0.9710 - accuracy: 0.6654 - val_loss: 0.8379 - val_accuracy: 0.7

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

train_hist = model3.fit(train_images, train_labels, 
                   validation_data=(val_images, val_labels), # or use `validation_split=0.1`
                   epochs=50)


Epoch 1/50


2023-02-20 22:06:45.466937: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout_4/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1563/1563 [==============================] - 25s 16ms/step - loss: 0.4410 - accuracy: 0.8511 - val_loss: 11.0006 - val_accuracy: 0.1024
Epoch 2/50
1563/1563 [==============================] - 24s 15ms/step - loss: 0.4232 - accuracy: 0.8560 - val_loss: 13.0379 - val_accuracy: 0.1024
Epoch 3/50
1563/1563 [==============================] - 24s 15ms/step - loss: 0.4092 - accuracy: 0.8606 - val_loss: 11.7869 - val_accuracy: 0.1026
Epoch 4/50
1563/1563 [==============================] - 24s 15ms/step - loss: 0.4080 - accuracy: 0.8623 - val_loss: 10.3241 - val_accuracy: 0.1026
Epoch 5/50
1563/1563 [==============================] - 24s 15ms/step - loss: 0.4032 - accuracy: 0.8626 - val_loss: 10.9090 - val_accuracy: 0.1024
Epoch 6/50
1563/1563 [==============================] - 24s 15ms/step - loss: 0.3949 - accuracy: 0.8656 - val_loss: 13.7157 - val_accuracy: 0.1020
Epoch 7/50
1563/1563 [==============================] - 24s 15ms/step - loss: 0.3908 - accuracy: 0.8663 - val_loss: 11.6928 - val

In [ ]:
def test_model3_params():
  assert model3.count_params() in [709066, 708554]

def test_model3_adds():
  n_adds = count_layers(model3, "Add")
  print(n_adds)
  assert n_adds == 3

def count_layers(model, layer_type):
  lyr_count = 0
  for l in model.layers: 
    if l.__class__.__name__ == layer_type:
      lyr_count +=1 
  return lyr_count

In [ ]:
test_model3_adds()
test_model3_params()

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation loss
plt.plot(train_hist.history['loss'])
plt.plot(train_hist.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()

# Plot training and validation accuracy
plt.plot(train_hist.history['accuracy'])
plt.plot(train_hist.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='lower right')
plt.show()
